In [1]:
import os
import requests
import json
import pandas as pd
import numpy as np
import PyPDF2
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import random

from openai import OpenAI
from sentence_transformers import SentenceTransformer, losses, InputExample
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import torch
from torch.utils.data import DataLoader

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.finetuning import generate_qa_embedding_pairs, EmbeddingQAFinetuneDataset

In [2]:
model = SentenceTransformer('BAAI/bge-m3')

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
Load pretrained SentenceTransformer: BAAI/bge-m3


In [3]:
train_examples = [
    InputExample(texts=['AI란 무엇인가?', 'AI는 인간의 지능을 모방한 기술입니다.', 'AI는 로봇과 같은 물리적 형태를 가진 기계입니다.']),
    InputExample(texts=['딥러닝이란?', '신경망을 여러 층 쌓아 데이터로부터 학습하는 기계학습 방법입니다.', '컴퓨터가 스스로 생각하는 방법입니다.']),
    InputExample(texts=['Python은 어디에 쓰이나요?', 'Python은 데이터 분석, 웹 개발, AI 등에 널리 사용됩니다.']),
    InputExample(texts=['자연어 처리란?', '컴퓨터가 인간의 언어를 이해하고 처리하는 AI의 한 분야입니다.']),
]

In [4]:
batch_size = 32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

In [5]:
loss = losses.MultipleNegativesRankingLoss(model, scale=20)

train_loss = losses.MultipleNegativesRankingLoss(model)
warmup_steps = int(len(train_dataloader) * 0.1)

In [6]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs = 3,
    warmup_steps=warmup_steps,
    optimizer_params={'lr':1e-5},
    output_path='../../../exercisebook_large_data/korean_sentence-embedding-model'
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


INFO:sentence_transformers.SentenceTransformer:Save model to ../../../exercisebook_large_data/korean_sentence-embedding-model
Save model to ../../../exercisebook_large_data/korean_sentence-embedding-model


In [7]:
def extract_text_from_pdf(pdf_path):
    text_chunks = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text.strip():
                text = text.strip()
                if len(text) > 10:
                    text_chunks.append(text)
    
    return text_chunks

In [8]:
train_corpus = extract_text_from_pdf('data/ict_usa_2024.pdf')
print(f'학습 데이터 문서 개수: {len(train_corpus)}')

val_corpus = extract_text_from_pdf('data/ict_japan_2024.pdf')
print(f'검증 데이터 문서 개수: {len(val_corpus)}')

학습 데이터 문서 개수: 26
검증 데이터 문서 개수: 27


In [9]:
client = OpenAI()

def generate_queries(corpus, num_questions_per_chunk=2):
    all_queries = []
    all_positive_docs = []

    prompt_template = '''
다음은 참고할 내용입니다.

---------------------
{context_str}
---------------------

위 내용을 바탕으로 낼 수 있는 질문을 {num_questions_per_chunk}개 만들어 주세요.
질문만 작성하고 실제 정답이나 보기 등은 작성하지 않습니다.

해당 질문은 본문을 볼 수 없다고 가정합니다. 따라서 '위 본문을 바탕으로~' 라는 식의 질문은 할 수 없습니다.

질문은 아래와 같은 형식으로 번호를 나열하여 생성하십시오.

1. (질문)
2. (질문)
'''
    for text in tqdm(corpus):
        messages = [
            {'role':'system', 'content':'You are a helpful assistant that generates questions based on provided content.'},
            {'role':'user', 'content':prompt_template.format(context_str=text, num_questions_per_chunk=num_questions_per_chunk)}
        ]

        response = client.chat.completions.create(
            model='gpt-5-nano',
            messages=messages,
            temperature=0.5
        )

        result = response.choices[0].message.content.strip().split('\n')

        questions = []
        for line in result:
            if line.strip():
                parts = line.strip().split('. ', 1)
                if len(parts) > 1:
                    questions.append(parts[1])
                else:
                    questions.append(parts[0])
        
        questions = [question for question in questions if len(question) > 0]

        for question in questions:
            all_queries.append(question)
            all_positive_docs.append(text)
    
    return all_queries, all_positive_docs

In [10]:
usa_documents = SimpleDirectoryReader(input_files=['data/ict_usa_2024.pdf']).load_data()
japan_documents = SimpleDirectoryReader(input_files=['data/ict_japan_2024.pdf']).load_data()
node_parser = SentenceSplitter(chunk_size=512)
nodes_train = node_parser.get_nodes_from_documents(usa_documents)
nodes_valid = node_parser.get_nodes_from_documents(japan_documents)

node_count = 20
if len(nodes_train) > node_count:
    selected_nodes_train = random.sample(nodes_train, node_count)
else:
    selected_nodes_train = nodes_train

if len(nodes_valid) > node_count:
    selected_nodes_valid = random.sample(nodes_valid, node_count)
else:
    selected_nodes_valid = nodes_valid
print(len(selected_nodes_train), len(selected_nodes_valid))

llm = OpenAI(model="gpt-5-nano", temperature=0.0)

20 20


In [11]:
# 저장할 파일 이름
DATASET_PATH_TRAIN = "../../../exercisebook_large_data/ict_dataset/usa_train_dataset.json"

if os.path.exists(DATASET_PATH_TRAIN):
    # 1. 파일이 이미 있으면 -> 로드 (0.1초 컷)
    print(f"이미 저장된 데이터셋이 있습니다. '{DATASET_PATH_TRAIN}'를 로드합니다...")
    train_dataset = EmbeddingQAFinetuneDataset.from_json(DATASET_PATH_TRAIN)
    
else:
    # 2. 파일이 없으면 -> 생성 (30분 소요)
    print("저장된 데이터셋이 없습니다. 생성을 시작합니다... (시간 소요됨)")
    
    # (이미 node_parser 등을 통해 nodes가 준비되었다고 가정)
    train_dataset = generate_qa_embedding_pairs(
        nodes=selected_nodes_train,
        llm=llm,
        num_questions_per_chunk=2,
        output_path=DATASET_PATH_TRAIN  # 여기서 생성과 동시에 저장까지 됩니다
    )
    print("생성 및 저장 완료!")

# 이후 로직 진행
print(f"데이터 준비 완료: 쿼리 {len(train_dataset.queries)}개")

print(train_dataset.queries)
print(train_dataset.relevant_docs)

이미 저장된 데이터셋이 있습니다. '../../../exercisebook_large_data/ict_dataset/usa_train_dataset.json'를 로드합니다...
데이터 준비 완료: 쿼리 40개
{'9fa30f6f-3f65-4aeb-abea-cecd06b6e6c6': '서술형 질문: 문맥에 명시된 기업의 세 가지 주요 약속을 나열하고, 각 약속이 AI 시스템의 사회적 위험 관리에 어떻게 기여하는지 간단히 설명하시오.', 'e1bc43d9-7e32-440d-8fec-df5c0ff74bba': '객관식 질문: 이 문맥의 출처로 명시된 기관은 어디인가?', '701ca168-53ad-4cc9-967f-e1b864aee304': '다지선다: 다음 중 본문에서 언급된 양국 간 협력의 주요 목표로 올바르게 진술된 것은 무엇인가?', '5326e580-a7f1-4e13-8495-5d72251975bd': '- A) 사이버보안 표준과 관행의 발전', '8f86fa87-0b23-478d-af04-bd4d2d4be81d': '객관식 문제: 반도체 과학법(CHIPS and Science Act)의 주요 목표로 옳은 것은 무엇인가?', '64a7a156-acb0-4699-9ae3-5e0b378dec34': 'A) 미국 내 반도체 생산을 해외로 이전하는 것', 'a2ce1a64-e6c4-44a4-8f73-e440e54d3001': '다음 중 ‘대중의 신뢰 확보’ 원칙에 해당하는 구체적 조치를 모두 고르시오. (복수 정답 가능)', '9690541a-135f-4253-a9d8-1c0ae0009670': 'A. 콘텐츠의 AI 생성 시점을 사용자가 알 수 있도록 워터마킹 시스템 등 기술 메커니즘 개발', '84d0c0dc-9dcf-4d37-a67a-a66821c387b5': '다지선다형: 다음 중 FAA의 BVLOS 면제 부여가 드론 배송 시장에 기여하는 주요 효과로 올바른 것은 무엇인가?', 'a9b85780-9c3f-44b4-b8c8-a879759e00d2': 'A) 비

In [12]:
# 저장할 파일 이름
DATASET_PATH_VALID = "../../../exercisebook_large_data/ict_dataset/japan_valid_dataset.json"

if os.path.exists(DATASET_PATH_VALID):
    # 1. 파일이 이미 있으면 -> 로드 (0.1초 컷)
    print(f"이미 저장된 데이터셋이 있습니다. '{DATASET_PATH_VALID}'를 로드합니다...")
    valid_dataset = EmbeddingQAFinetuneDataset.from_json(DATASET_PATH_VALID)
    
else:
    # 2. 파일이 없으면 -> 생성 (30분 소요)
    print("저장된 데이터셋이 없습니다. 생성을 시작합니다... (시간 소요됨)")
    
    # (이미 node_parser 등을 통해 nodes가 준비되었다고 가정)
    valid_dataset = generate_qa_embedding_pairs(
        nodes=selected_nodes_valid,
        llm=llm,
        num_questions_per_chunk=2,
        output_path=DATASET_PATH_VALID  # 여기서 생성과 동시에 저장까지 됩니다
    )
    print("생성 및 저장 완료!")

# 이후 로직 진행
print(f"데이터 준비 완료: 쿼리 {len(valid_dataset.queries)}개")

print(valid_dataset.queries)
print(valid_dataset.relevant_docs)

저장된 데이터셋이 없습니다. 생성을 시작합니다... (시간 소요됨)


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  5%|▌         | 1/20 [00:26<08:21, 26.39s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 95%|█████████▌| 19/20 [07:07<00:21, 21.32s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 20/20 [07:37<00:00, 22.88s/it]

Final dataset saved.
생성 및 저장 완료!
데이터 준비 완료: 쿼리 40개
{'22ee7f6e-3c46-49da-ba06-5be386a88d07': '다음은 규슈 지역 등 제3·4의 핵심거점 정비와 관련된 목표로 제시된 내용이다. 이 중 목표에 포함되지 않는 것은 무엇인가?', 'f6458070-cc14-44af-9c14-9e17058fcd0f': 'A) 지역 분산형 데이터센터 건설', 'db2e1638-9f94-45aa-a89a-3ae3252a5c52': 'Multiple Choice:', '6542c0f0-dc50-4e1d-8144-1c8b14e85573': 'Based on the context, which regions are considered suitable for data centers due to renewable energy production, and what is the size and duration of the proposed subsidy plan?', '316cc49a-294b-4460-9578-0dbf29c460e6': '다지선다형: 일본의 최초 레벨 4 자율주행 모빌리티 서비스에 대해 옳은 진술을 고르시오.', '3ec23343-e1e6-4a69-bac5-e8a3ccb836d3': 'A) Level 2', '50b94b49-e4cc-4d1f-a0ec-ae230e94b95a': 'Question 1 (단답형/서술형):', '6175a142-e311-4762-94d6-1a58a461e8c0': '다음 문단에 기술된 일본의 METI(경제산업성)의 ICT 과학기술 관련 역할을 세 가지 이상 서술하시오.', '88f22a28-b03d-4f4a-8867-8ec6cfbe11cd': '다지선다형: 아래 보기 중 문맥에서 언급된 일본의 Web3 산업 촉진 관련 내용이 아닌 것은 무엇인가?', '6ecd9ade-0a93-47de-a668-ad63bb30a84c': '- A) NFT 규제 제안', '676abee3-362a-4e2e-